<a href="https://colab.research.google.com/github/BasitRauf/AssignmentsPIAIC/blob/main/assignment_2_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.9 MB/s eta 0:00:00


In [ ]:
from pinecone import Pinecone,ServerlessSpec
import os
from google.colab import userdata
os.environ["PINECONE_API_KEY"] = userdata.get('PINECONE_API_KEY')
pc = Pinecone()


In [ ]:
index_name = "first-rag-project"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    print(f"Index '{index_name}' created successfully.")
else:
    print(f"Index '{index_name}' already exists.")

index = pc.Index(index_name)

Index 'first-rag-project' already exists.


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from google.auth.credentials import AnonymousCredentials
import os

credentials = userdata.get('GOOGLE_API_KEY')

os.environ['GOOGLE_API_KEY'] = credentials

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=credentials
)

vector = embeddings.embed_query("we are building a RAG Text")

print(vector[:5])

[0.04060795158147812, -0.04385491460561752, -0.05683692544698715, -0.021194666624069214, 0.010449625551700592]


In [ ]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(
    index=index,
    embedding=embeddings
    )


In [ ]:
from langchain_core.documents import Document
from uuid import uuid4
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]



In [ ]:
len(documents)

5

In [ ]:
# user_id = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents)


['12b20adf-28e1-4a4b-a45c-829bfe19cd9f',
 '353c13c4-c372-456b-9c3a-5af1fab1c372',
 '2ca68f3d-40e9-41b6-b0f0-dd7de9b2f4f8',
 '33eebf8f-6f45-42c0-bd6c-f7edb4b24bd9',
 'aa9ba9ca-a551-42c2-bff0-30c367d65cad']

In [ ]:
results = vector_store.similarity_search_with_score(
    "who won the election of us?",)
results
# for result,score in results:
#   print(result,score)
  # print(result,similarity_search_with_score)
# results
# results = [1,[3, 5, 7, 2, 8, 10, 1]]
# highest = results[1][1]
# # highest
# for result in range(len(results)):
#   print(results[result][1])
#   if results[result][1] > highest:
#     highest = results[result][1]
# highest

#     if results[1][1] > highest:
#         highest = result
# highest


[(Document(id='8789aae8-7636-4d7a-9c6d-e3a6d04c7a84', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  0.437263608),
 (Document(id='12b20adf-28e1-4a4b-a45c-829bfe19cd9f', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  0.436524481),
 (Document(id='6f438f1c-dd1c-47e4-8105-f1280af05df3', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  0.405955076),
 (Document(id='33eebf8f-6f45-42c0-bd6c-f7edb4b24bd9', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  0.405058354)]